<center><img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px; margin-left: 10px; max-height: 250px;" /></center>

# 2.3 Data Access and Basic Processing


<img src="https://openeo.org/images/openeo_logo.png"
     alt="openEO logo"
     style="float: center; margin-right: 10px; max-height: 100px;"/>
     

## Resample Operators with openEO

Sometimes we need to align the spatial or temporal dimension of two datacubes, so that they can be merged together.

### `resample_cube_spatial`: spatial resampling Sentinel-2 to match Landsat

Start importing the necessary libraries and initialize a local connection for Client-Side Processing.

In [1]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

Create two datacubes, one for Sentinel-2 and one for Landsat

In [2]:
spatial_extent = {"west": 11.4, "east": 11.42, "south": 45.5, "north": 45.52}
temporal_extent = ["2023-06-01", "2023-06-30"]

Datacube for Sentinel-2

In [3]:
url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"
bands = ["red","nir"]
s2_cube = local_conn.load_stac(url=url,
   spatial_extent=spatial_extent,
   temporal_extent=temporal_extent,
   bands=bands
)
s2_cube.execute()

<xarray.DataArray 'stackstac-9ccd78cc6884f8c7da8c56505a6169c0' (time: 6,
                                                                band: 2,
                                                                y: 227, x: 163)> Size: 4MB
dask.array<getitem, shape=(6, 2, 227, 163), dtype=float64, chunksize=(1, 1, 227, 163), chunktype=numpy.ndarray>
Coordinates: (12/54)
  * time                                     (time) datetime64[ns] 48B 2023-0...
    id                                       (time) <U24 576B 'S2A_32TPR_2023...
  * band                                     (band) <U3 24B 'red' 'nir'
  * x                                        (x) float64 1kB 6.874e+05 ... 6....
  * y                                        (y) float64 2kB 5.044e+06 ... 5....
    s2:medium_proba_clouds_percentage        (time) float64 48B 16.89 ... 18.45
    ...                                       ...
    proj:shape                               object 8B {10980}
    proj:transform                           object 8B {0, 5100000, 600000, 1...
    common_name                              (band) <U3 24B 'red' 'nir'
    center_wavelength                        (band) float64 16B 0.665 0.842
    full_width_half_max                      (band) float64 16B 0.038 0.145
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 4990200.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5100000.00|\n| 0.0...
    resolution:  10.0

Datacube for Landsat

In [4]:
url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2"
bands = ["red","nir08"]
l8_cube = local_conn.load_stac(url=url,
                    spatial_extent=spatial_extent,
                    temporal_extent=temporal_extent,
                    bands=bands)
l8_cube.execute()

<xarray.DataArray 'stackstac-ffed4b86d01e4e9a000a39482ddbf8cd' (time: 15,
                                                                band: 2, y: 75,
                                                                x: 54)> Size: 972kB
dask.array<getitem, shape=(15, 2, 75, 54), dtype=float64, chunksize=(1, 1, 75, 54), chunktype=numpy.ndarray>
Coordinates: (12/27)
  * time                         (time) datetime64[ns] 120B 2023-06-03T10:03:...
    id                           (time) <U31 2kB 'LC08_L2SP_193028_20230603_0...
  * band                         (band) <U5 40B 'red' 'nir08'
  * x                            (x) float64 432B 6.874e+05 ... 6.89e+05
  * y                            (y) float64 600B 5.044e+06 ... 5.041e+06
    proj:epsg                    int64 8B 32632
    ...                           ...
    landsat:collection_number    <U2 8B '02'
    created                      (time) <U27 2kB '2023-06-09T09:14:58.222209Z...
    platform                     (time) <U9 540B 'landsat-8' ... 'landsat-8'
    raster:bands                 object 8B {'scale': 2.75e-05, 'nodata': 0, '...
    title                        (band) <U22 176B 'Red Band' 'Near Infrared B...
    epsg                         int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(495570.0, 4825470.0, 855630.0...
    crs:         epsg:32632
    transform:   | 30.00, 0.00, 495570.00|\n| 0.00,-30.00, 5217630.00|\n| 0.0...
    resolution:  30.0

From the previous outputs, notice the shape difference in the spatial dimensions `x` and `y`.

This is due to the different resolution of the two collections: 10m for Sentinel-2, 30m for Landsat.

Now we use the `resample_cube_spatial` process to resample the Sentinel-2 data to match Landsat.

In [5]:
s2_cube_30m = s2_cube.resample_cube_spatial(target=l8_cube,method="average")
s2_cube_30m

Check what happens to the datacube inspecting the resulting xArray object. Now the `x` and `y` shape is the same as Landsat:

In [6]:
s2_cube_30m.execute()

<xarray.DataArray 'reproject-8bc8161df53d46159228f9910b3bdb95' (time: 6,
                                                                band: 2, y: 75,
                                                                x: 54)> Size: 389kB
dask.array<reproject, shape=(6, 2, 75, 54), dtype=float64, chunksize=(1, 1, 75, 54), chunktype=numpy.ndarray>
Coordinates: (12/55)
  * time                                     (time) datetime64[ns] 48B 2023-0...
    id                                       (time) <U24 576B 'S2A_32TPR_2023...
  * band                                     (band) <U3 24B 'red' 'nir'
    s2:medium_proba_clouds_percentage        (time) float64 48B 16.89 ... 18.45
    s2:nodata_pixel_percentage               int64 8B 0
    processing:software                      object 8B {'sentinel2-to-stac': ...
    ...                                       ...
    center_wavelength                        (band) float64 16B 0.665 0.842
    full_width_half_max                      (band) float64 16B 0.038 0.145
    epsg                                     int64 8B 32632
  * y                                        (y) float64 600B 5.044e+06 ... 5...
  * x                                        (x) float64 432B 6.874e+05 ... 6...
    spatial_ref                              int64 8B 0
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 4990200.0, 709800.0...
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5100000.00|\n| 0.0...
    resolution:  10.0